#### Dermatology Model (WIP)

Descrição: O dataset Dermatology contém atributos clínicos de pacientes para diagnosticar seis tipos diferentes de eritroderma. As classes são os seis tipos de doença de pele, numeradas de 1 a 6 e tem 366 instâncias no total.

### Imports

In [19]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


### Carregando o Dataset

In [25]:
car_evaluation = fetch_ucirepo(id=33)
X_original = car_evaluation.data.features
y_original = car_evaluation.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
#print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
#print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")


Dimensão inicial de X: (366, 34)

Classes de y:
class
1    112
3     72
2     61
5     52
4     49
6     20
Name: count, dtype: int64


### Binarização dos Atributos para o WiSARD usando o Termômetro

In [21]:
from module import termometro

X_encoded_parts = []
n_bits = 8 

for col in X_original.columns:
    X_encoded_parts.append(termometro.codificador_termometro(X_original[col].astype(float), n_bits))
    #print(f"Coluna '{col}' binarizada com {n_bits} bits (Termômetro).")

X_final_binary = np.hstack(X_encoded_parts)
X_encoded_uint8 = X_final_binary.astype(np.uint8)
print(X_encoded_uint8)

[[1 1 1 ... 1 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 1 0 0]
 [1 1 1 ... 0 0 0]]


### Dividindo os Dados em Treino e Teste

In [22]:
y_str = y_original.astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_uint8, y_str, test_size=0.7, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")


Tamanho do conjunto de treino (X_train): 109 amostras, 272 bits
Tamanho do conjunto de teste (X_test): 257 amostras, 272 bits
Tipo dos elementos de X_train: uint8


### Treinamento do Modelo WiSARD

In [23]:
addressSize = 16 

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=False)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)

predictions_str = wsd.classify(X_test)

y_pred_np = np.array(predictions_str)
y_test_np = np.array(y_test)

WiSARD inicializado com addressSize=16


### Relatório de Classificação

In [24]:
print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test)
))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        79
           2       0.87      0.93      0.90        43
           3       1.00      0.98      0.99        51
           4       0.91      0.85      0.88        34
           5       0.97      1.00      0.99        36
           6       1.00      0.93      0.96        14

    accuracy                           0.96       257
   macro avg       0.96      0.95      0.95       257
weighted avg       0.96      0.96      0.96       257



### Conclusões

O modelo WiSARD performou muito bem no dataset, alcançando números significativos para as métricas de avaliação.